In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences

In [24]:
data = pd.read_csv('text2sql.csv')
len(data)

3072

#### Data Cleanup

In [25]:
# ToLower

data['seq1'] = data['seq1'].str.lower()
data['seq2'] = data['seq2'].str.lower()
data['seq3'] = data['seq3'].str.lower()
data['sql1'] = data['sql1'].str.lower()
data['sql2'] = data['sql2'].str.lower()
data['sql3'] = data['sql3'].str.lower()

# Remove '?'
data['seq1'] = data['seq1'].str.replace('?','')
data['seq2'] = data['seq2'].str.replace('?','')
data['seq3'] = data['seq3'].str.replace('?','')
data['sql1'] = data['sql1'].str.replace('?','')
data['sql2'] = data['sql2'].str.replace('?','')
data['sql3'] = data['sql3'].str.replace('?','')

# Append start_ targetsql _end

data['sql1'] = 'start_ '+data['sql1'].astype(str)+' _end'
data['sql2'] = 'start_ '+data['sql2'].astype(str)+' _end'
data['sql3'] = 'start_ '+data['sql3'].astype(str)+' _end'

In [26]:
# Input_Texts = Natural Language Text Sequence
# Target_Texts = Natural Language SQL Sequence

input_texts = []
target_texts = []

for column in data[['seq1', 'seq2', 'seq3']]:
    columnobj = data[column]
    input_texts.extend(columnobj.values.tolist())
    
for column in data[['sql1', 'sql2', 'sql3']]:
    columnobj = data[column]
    target_texts.extend(columnobj.values.tolist())
    
# input_texts,target_texts

#### BOW Representation

In [27]:
all_seq_words=set()
all_sql_words= set()

length_list_seq = []
length_list_sql = []

sequence = data.seq1

for l in sequence:
    length_list_seq.append(len(l.split()))
#     print(l.split())
#     print(length_list)
# print(np.max(length_list))
for eng in sequence:
    for word in eng.split():
        if word not in all_seq_words:
            all_seq_words.add(word)

sequence = data.seq2 

for l in sequence:
    length_list_seq.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_seq_words:
            all_seq_words.add(word)
            
print("Maximum SEQ Length",np.max(length_list_seq))
max_seq_length = np.max(length_list_seq)

sequence = data.seq3

for l in sequence:
    length_list_seq.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_seq_words:
            all_seq_words.add(word)
    

sequence = data.sql1 

for l in sequence:
    length_list_sql.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_sql_words:
            all_sql_words.add(word)

sequence = data.sql2 

for l in sequence:
    length_list_sql.append(len(l.split()))
    
for eng in sequence:
    for word in eng.split():
        if word not in all_sql_words:
            all_sql_words.add(word)

sequence = data.sql3 

for l in sequence:
    length_list_sql.append(len(l.split()))

for eng in sequence:
    for word in eng.split():
        if word not in all_sql_words:
            all_sql_words.add(word)
                        
print("Maximum SQL Length",np.max(length_list_sql))
max_sql_length = np.max(length_list_sql)

Maximum SEQ Length 8
Maximum SQL Length 16


In [28]:
input_words = sorted(list(all_seq_words))
target_words = sorted(list(all_sql_words))
num_encoder_tokens = len(all_seq_words)
num_decoder_tokens = len(all_sql_words)

max_sql_length = np.max(length_list_sql)

max_encoder_seq_length = max_seq_length # max([len(txt) for txt in input_texts])
max_decoder_seq_length = max_sql_length#max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 9216
Number of unique input tokens: 36
Number of unique output tokens: 42
Max sequence length for inputs: 8
Max sequence length for outputs: 16


In [29]:
print(input_words)
print(len(input_words))

['a', 'count', 'day', 'days', 'eight', 'eigth', 'eleven', 'five', 'for', 'forty', 'four', 'hour', 'is', 'last', 'me', 'my', 'nine', 'ninty', 'one', 'past', 'performance', 'seven', 'seventy', 'show', 'six', 'status', 'ten', 'three', 'total', 'trend', 'tweleve', 'twenty', 'two', 'walk', 'week', 'what']
36


In [30]:
print(target_words)
print(len(target_words))

['_end', 'aggregate', 'and', 'consecutive', 'day', 'days', 'eight', 'eigth', 'eigthhour', 'eigthhours', 'eleven', 'equal', 'equalt', 'five', 'forty', 'four', 'hour', 'hours', 'is', 'last', 'nine', 'ninty', 'of', 'one', 'past', 'select', 'selection', 'seven', 'seventy', 'six', 'sixty', 'start_', 'sum', 'ten', 'three', 'to', 'tweleve', 'twenty', 'two', 'walk', 'week', 'where']
42


#### Assigning each unique word to an integer value.

In [31]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])
# input_token_index,target_token_index

In [32]:
len(input_token_index),len(target_token_index)

(36, 42)

In [33]:
# input_token_index,target_token_index

In [34]:
len(input_texts),len(target_texts)

(9216, 9216)

In [35]:
num_encoder_tokens,num_decoder_tokens

(36, 42)

In [36]:
max_encoder_seq_length,max_decoder_seq_length

(8, 16)

In [37]:
total_length_sql = len(data['sql1']) + len(data['sql2']) + len(data['sql3'])
total_length_seq = len(data['seq1']) + len(data['seq2']) + len(data['seq3'])

In [38]:
encoder_input_data = np.zeros((total_length_seq, max_seq_length),dtype='float32')
decoder_input_data = np.zeros((total_length_sql, max_sql_length),dtype='float32')
decoder_target_data = np.zeros((total_length_sql, max_sql_length, num_decoder_tokens),dtype='float32')

In [39]:
# import numpy as np

# encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length),dtype='float32')
# decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length),dtype='float32')
# decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [40]:
encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape

((9216, 8), (9216, 16), (9216, 16, 42))

In [41]:
num_encoder_tokens,num_decoder_tokens

(36, 42)

* The encoder_input_data will consist of  [144] samples of the maximum sequence length (40) filled with the respective one-hot-encoded tokens (in this case a vector of length 36).

In [42]:
# code reference - https://nextjournal.com/gkoehler/machine-translation-seq2seq-cpu
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
#     print(i,input_text,"-->",target_text)
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [43]:
# encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_encoder_seq_length,padding='pre')
# encoder_input_data = np.flip(encoder_input_data,axis = 1)
# decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_decoder_seq_length)

In [44]:
batch_size = 32  # batch size for training
epochs = 5  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

rmsprop = RMSprop(learning_rate=0.001, rho=0.9)

### Encoder

In [45]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

### Decoder

In [46]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, latent_dim)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [47]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    9216        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    10752       input_2[0][0]                    
____________________________________________________________________________________________

In [48]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.05)
model.save('seq2HealthSQL.h5')

C:\Users\vijay\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8755 samples, validate on 461 samples
Epoch 1/5
8755/8755 [==============================] - 24s 3ms/step - loss: 0.5358 - acc: 0.8383 - val_loss: 0.2693 - val_acc: 0.9012
Epoch 2/5
8755/8755 [==============================] - 24s 3ms/step - loss: 0.1974 - acc: 0.9228 - val_loss: 0.1649 - val_acc: 0.9261
Epoch 3/5
8755/8755 [==============================] - 25s 3ms/step - loss: 0.1345 - acc: 0.9387 - val_loss: 0.1449 - val_acc: 0.9454
Epoch 4/5
8755/8755 [==============================] - 24s 3ms/step - loss: 0.0765 - acc: 0.9545 - val_loss: 0.0423 - val_acc: 0.9653
Epoch 5/5
8755/8755 [==============================] - 24s 3ms/step - loss: 0.0430 - acc: 0.9524 - val_loss: 0.0297 - val_acc: 0.9658


In [49]:
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.load_weights(seq2HealthSQL.h5)

In [50]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)

In [51]:
# reverse-lookup token index to turn sequences back to characters
reverse_input_word_index = dict(
  (i, word) for word, i in input_token_index.items())
reverse_target_word_index = dict(
  (i, word) for word, i in target_token_index.items())

In [52]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['start_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_end' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [53]:
encoder_input_data.shape

(9216, 8)

In [54]:
for seq_index in range(50):
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
#     input_seq = pad_sequences(input_seq, maxlen=max_encoder_seq_length)
    
#     print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
#     print(decoded_sentence)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: past hour walk status 
Decoded sentence:  select walk where
-
Input sentence: last hour walk status 
Decoded sentence:  select walk where
-
Input sentence: past two hour walk status 
Decoded sentence:  select walk where
-
Input sentence: last two hour walk status 
Decoded sentence:  select walk where
-
Input sentence: past three hour walk status 
Decoded sentence:  select walk where
-
Input sentence: last three hour walk status 
Decoded sentence:  select walk where
-
Input sentence: past four hour walk status 
Decoded sentence:  select walk where
-
Input sentence: last four hour walk status 
Decoded sentence:  select walk where
-
Input sentence: past five hour walk status 
Decoded sentence:  select walk where
-
Input sentence: last five hour walk status 
Decoded sentence:  select walk where
-
Input sentence: past six hour walk status 
Decoded sentence:  select walk where
-
Input sentence: last six hour walk status 
Decoded sentence:  select walk where
-
Input sentence

In [55]:
input_sentence = "last six hour walk status"

test_sentence_tokenized = np.zeros((len(input_texts), max_encoder_seq_length),dtype='float32')
print(test_sentence_tokenized.shape)

for t, word in enumerate(input_sentence.split()):
    print(input_token_index[word])
    test_sentence_tokenized[0, t] = input_token_index[word]
    
print("\n")   
print(input_sentence,"-->\n",decode_sequence(test_sentence_tokenized))

(9216, 8)
13
24
11
33
25


last six hour walk status -->
  select walk where
